In [1]:
#using PyCall
#np = pyimport("numpy")
#using DifferentialEquations
#using DynamicalSystems
#using PyPlot
#using DiffEqCallbacks
#using ForwardDiff
#using Latexify, LaTeXStrings, SymEngine
#using LinearAlgebra
#using JLD

In [2]:
#include("Utils\\Computation_Utils.jl")
#include("Utils\\CallBack_Utils.jl")
#include("Utils\\YAML_Utils.jl")
#include("Utils\\Graphics_Utils.jl")
include("ODEs\\BS_Uni_System.jl")
#include("Utils\\TangentComputation_Utils.jl")

Main.BS_Uni_Sys

In [23]:
function Funcs_Compare(ReducedSys, DefaultSys, SyncWith)
    # Разница между первыми тремя элементами (phi) полной 6D системы и редуцированной 3D системой при условии предполагаемой синхронизации phi_i ~ psi_j + delay_j
    PhiDiffMax = 0

    # Разница между первыми тремя элементами (phi) и последними тремя элементами (psi) полной 6D системы при условии предполагаемой синхронизации phi_i ~ psi_j + delay_j
    PsiDiffMax = 0

    for i = 1 : 1000

        ReducedIC = rand(Float64, 3)
        DefaultIC = cat(ReducedIC, 
                        ReducedIC[SyncWith["phi1"]["psi"]] + SyncWith["phi1"]["delay"],
                        ReducedIC[SyncWith["phi2"]["psi"]] + SyncWith["phi2"]["delay"],
                        ReducedIC[SyncWith["phi3"]["psi"]] + SyncWith["phi3"]["delay"], dims = 1)

        RS = ReducedSys(ReducedIC)
        DS = DefaultSys(DefaultIC)

        Phidiff = maximum(map(abs, RS - DS[1 : 3]))
        if Phidiff > PhiDiffMax
            PhiDiffMax = Phidiff
        end

        Psidiff = maximum(map(abs, DS[1 : 3] - DS[3 .+ [SyncWith["phi1"]["psi"], SyncWith["phi2"]["psi"], SyncWith["phi3"]["psi"]]]))
        if Psidiff > PsiDiffMax
            PsiDiffMax = Psidiff
        end
    end
    return (PhiDiffMax, PsiDiffMax)
end

function makeSync(psi; delay = [0.0, 0.0, 0.0])
    SyncWith = Dict("phi1" => Dict("psi" => psi[1], "delay" => delay[1]), 
                    "phi2" => Dict("psi" => psi[2], "delay" => delay[2]),
                    "phi3" => Dict("psi" => psi[3], "delay" => delay[3]))
    println("Sync: phi_1 -> psi_", string(SyncWith["phi1"]["psi"]), " + ",  string(SyncWith["phi1"]["delay"]), "    ", 
              "phi_2 -> psi_", string(SyncWith["phi2"]["psi"]), " + ",  string(SyncWith["phi2"]["delay"]), "    ",
              "phi_3 -> psi_", string(SyncWith["phi3"]["psi"]), " + ",  string(SyncWith["phi3"]["delay"]))
    return SyncWith
end

makeSync (generic function with 1 method)

In [4]:
CoupleFuncNum = 2

if CoupleFuncNum == 1
    
    function MyCoupleFunc(x)
        return 1/(1 + exp(10*(cos(x))))
    end
    CoupleFuncName = "1_(1 + exp(10(cos(x))))"
     
elseif CoupleFuncNum == 2
    
    function MyCoupleFunc(x)
        return -cos(x)
    end

    CoupleFuncName = "-cos(x)"
    
elseif CoupleFuncNum == 3
    
    function MyCoupleFunc(x)
        return 1-cos(x)
    end

    CoupleFuncName = "1-cos(x)"
    
end

"-cos(x)"

In [5]:
a2 = pi/2
a4 = 3.24685581148453
K = 1.5789473684210527
r = K/4

Eps = 1e-7

config = Dict("a2" => a2, "a4" => a4, "K" => K, "r" => r, "Eps" => Eps)
DefaultSys = x -> BS_Uni_Sys.Get_Uni_Sys(config, MyCoupleFunc)(zeros(6), x, [], 0)

#2 (generic function with 1 method)

---

In [15]:
function proofSyncs(Syncs, Delays)
    for sync in Syncs
        SyncWith = makeSync(sync)
        ReducedSys = x -> BS_Uni_Sys.Get_Reduced_Sys(config, MyCoupleFunc, SyncWith)(zeros(3), x, [], 0);

        diff = Funcs_Compare(ReducedSys, DefaultSys, SyncWith)
        println("max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = ", string(diff[1]))
        println("max(DefaultSys[1 : 3] - DefaultSys[4 : 6]) = ", string(diff[2]))
        println("###")
    end
end

proofSyncs (generic function with 2 methods)

In [16]:
Syncs = [[1, 2, 3], [3, 1, 2], [2, 3, 1], [1, 3, 2], [3, 2, 1], [2, 1, 3]]
Delays = [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]
DelaysPi = [[pi for p in 1:3] for k in 1 : 6]

6-element Vector{Vector{Irrational{:π}}}:
 [π, π, π]
 [π, π, π]
 [π, π, π]
 [π, π, π]
 [π, π, π]
 [π, π, π]

In [17]:
proofSyncs(Syncs, Delays)

Sync: phi_1 -> psi_1 + 0.0    phi_2 -> psi_2 + 0.0    phi_3 -> psi_3 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
max(DefaultSys[1 : 3] - DefaultSys[4 : 6]) = 1.1732795294752951
###
Sync: phi_1 -> psi_3 + 0.0    phi_2 -> psi_1 + 0.0    phi_3 -> psi_2 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
max(DefaultSys[1 : 3] - DefaultSys[4 : 6]) = 0.7716575469841842
###
Sync: phi_1 -> psi_2 + 0.0    phi_2 -> psi_3 + 0.0    phi_3 -> psi_1 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
max(DefaultSys[1 : 3] - DefaultSys[4 : 6]) = 1.316503943389054
###
Sync: phi_1 -> psi_1 + 0.0    phi_2 -> psi_3 + 0.0    phi_3 -> psi_2 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
max(DefaultSys[1 : 3] - DefaultSys[4 : 6]) = 4.440892098500626e-16
###
Sync: phi_1 -> psi_3 + 0.0    phi_2 -> psi_2 + 0.0    phi_3 -> psi_1 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
max(DefaultSys[1 : 3] - DefaultSys[4 : 6]) = 4.440892098500626e-16
###
Sync: phi_1 -> psi_2 + 0.0    phi_2 -> psi_1 

In [18]:
proofSyncs(Syncs, DelaysPi)

Sync: phi_1 -> psi_1 + 0.0    phi_2 -> psi_2 + 0.0    phi_3 -> psi_3 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
max(DefaultSys[1 : 3] - DefaultSys[4 : 6]) = 1.1740444836271848
###
Sync: phi_1 -> psi_3 + 0.0    phi_2 -> psi_1 + 0.0    phi_3 -> psi_2 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
max(DefaultSys[1 : 3] - DefaultSys[4 : 6]) = 0.7721761944995027
###
Sync: phi_1 -> psi_2 + 0.0    phi_2 -> psi_3 + 0.0    phi_3 -> psi_1 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
max(DefaultSys[1 : 3] - DefaultSys[4 : 6]) = 1.3128582740374408
###
Sync: phi_1 -> psi_1 + 0.0    phi_2 -> psi_3 + 0.0    phi_3 -> psi_2 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
max(DefaultSys[1 : 3] - DefaultSys[4 : 6]) = 4.440892098500626e-16
###
Sync: phi_1 -> psi_3 + 0.0    phi_2 -> psi_2 + 0.0    phi_3 -> psi_1 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
max(DefaultSys[1 : 3] - DefaultSys[4 : 6]) = 4.440892098500626e-16
###
Sync: phi_1 -> psi_2 + 0.0    phi_2 -> psi_1

In [19]:
FullTanSys = x -> BS_Uni_Sys.Get_Tan_Full_Sys(config, MyCoupleFunc)(zeros(6), ones(6), x)

#22 (generic function with 1 method)

In [25]:
function Jacs_Compare(ReducedSys, DefaultSys, SyncWith)
    WDiffMax = 0

    for i = 1 : 1000

        ReducedIC = rand(Float64, 3)
        DefaultIC = cat(ReducedIC, 
                        ReducedIC[SyncWith["phi1"]["psi"]] + SyncWith["phi1"]["delay"],
                        ReducedIC[SyncWith["phi2"]["psi"]] + SyncWith["phi2"]["delay"],
                        ReducedIC[SyncWith["phi3"]["psi"]] + SyncWith["phi3"]["delay"], dims = 1)

        RS = ReducedSys(ReducedIC)
        DS = DefaultSys(DefaultIC)

        Phidiff = maximum(map(abs, RS - DS))
        if Phidiff > WDiffMax
            WDiffMax = Phidiff
        end
    end
    return WDiffMax
end

Jacs_Compare (generic function with 1 method)

In [28]:
    for sync in Syncs
        SyncWith = makeSync(sync)
        TanSys = x -> BS_Uni_Sys.Get_Tan_Red_Sys(config, MyCoupleFunc, SyncWith)(zeros(6), ones(6), x);

        diff = Jacs_Compare(TanSys, FullTanSys, SyncWith)
        println("max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = ", string(diff))
        println("###")
    end

Sync: phi_1 -> psi_1 + 0.0    phi_2 -> psi_2 + 0.0    phi_3 -> psi_3 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
###
Sync: phi_1 -> psi_3 + 0.0    phi_2 -> psi_1 + 0.0    phi_3 -> psi_2 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
###
Sync: phi_1 -> psi_2 + 0.0    phi_2 -> psi_3 + 0.0    phi_3 -> psi_1 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
###
Sync: phi_1 -> psi_1 + 0.0    phi_2 -> psi_3 + 0.0    phi_3 -> psi_2 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
###
Sync: phi_1 -> psi_3 + 0.0    phi_2 -> psi_2 + 0.0    phi_3 -> psi_1 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
###
Sync: phi_1 -> psi_2 + 0.0    phi_2 -> psi_1 + 0.0    phi_3 -> psi_3 + 0.0
max(ReducedSys[1 : 3] - DefaultSys[1 : 3]) = 0
###


# Library importing benchmark

In [1]:
@time using PyCall

  0.643947 seconds (742.23 k allocations: 41.900 MiB, 2.29% gc time, 71.95% compilation time)


In [2]:
@time import DifferentialEquations: SciMLBase.ODEProblem

 17.847246 seconds (41.41 M allocations: 3.005 GiB, 6.43% gc time, 16.77% compilation time: 86% of which was recompilation)


In [2]:
@time using SciMLBase

  0.318781 seconds (720.91 k allocations: 35.768 MiB, 5.73% gc time, 99.72% compilation time: 100% of which was recompilation)


In [1]:
@time using DiffEqCallbacks

  6.942304 seconds (14.33 M allocations: 1.069 GiB, 6.70% gc time, 14.14% compilation time: 60% of which was recompilation)


In [3]:
@time using DynamicalSystems

 16.451421 seconds (42.19 M allocations: 2.582 GiB, 2.91% gc time, 14.42% compilation time: 80% of which was recompilation)


In [2]:
@time using PyPlot

  3.887290 seconds (7.36 M allocations: 381.650 MiB, 3.19% gc time, 73.34% compilation time: 62% of which was recompilation)


In [5]:
@time using DiffEqCallbacks

  0.000768 seconds (279 allocations: 18.102 KiB)


In [6]:
@time using ForwardDiff

  0.000807 seconds (279 allocations: 18.102 KiB)


In [7]:
@time using Latexify, LaTeXStrings, SymEngine

  0.364160 seconds (609.21 k allocations: 47.282 MiB, 59.16% compilation time: 74% of which was recompilation)


In [8]:
@time using LinearAlgebra

  0.001835 seconds (654 allocations: 48.570 KiB)


In [3]:
using PackageCompiler

In [8]:
create_sysimage(["PyPlot"], sysimage_path="sys_plots.so")

- PackageCompiler: compiling incremental system image
